# Use SurvivalEVAL for models that only predicts times (until an event occurs)
In this notebook, we will use the SurvivalEVAL package to evaluate the performance of a model that only predicts the time until an event occurs. 

We will use the censored support vector regressor (cSVR) model as an example. The cSVR model is a survival model that uses support vector machines to predict the time until an event occurs. The model is implemented in the `SciKit-Survival` package.

In [ ]:
## Import the necessary packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sksurv.datasets import load_gbsg2
from sksurv.svm import FastKernelSurvivalSVM
from sksurv.preprocessing import OneHotEncoder, encode_categorical
from sksurv.metrics import check_y_survival

from Evaluator import PointEvaluator

In [ ]:
%matplotlib inline

## Load the dataset

In [ ]:
X, y = load_gbsg2()
Xt = encode_categorical(X)
X.loc[:, "tgrade"] = X.loc[:, "tgrade"].map(len).astype(int)
Xt = OneHotEncoder().fit_transform(X)

In [ ]:
n = len(Xt)

tr_size = int(n*0.70)

Xt_train, Xt_test = Xt.iloc[:tr_size, :], Xt.iloc[tr_size:, :]
y_train, y_test = y[:tr_size], y[tr_size:]
train_event_indicators, train_event_times = check_y_survival(y_train)
test_event_indicators, test_event_times = check_y_survival(y_test)

## Build the model and train it

In [ ]:
cSVR = FastKernelSurvivalSVM().fit(Xt_train, y_train)

## Predict the time until an event occurs

In [ ]:
t_pred = cSVR.predict(Xt_test)

## Evaluate the model
Some arguments includes:


In [ ]:
evaler = PointEvaluator(t_pred, test_event_times, test_event_indicators, train_event_times, train_event_indicators)